# clojure中的数据结构

clojure提供四种基本的集合数据(collection)类型。
- **list**: 以小括号包围的数据，比如`(1 2 3 4)`
- **vector**: 以中括号包围的数据，比如`[1 2 3 4]`
- **map**: 以大括号包围的键值对，比如`{'a' b}`
- **set**: 以`#{}`包围的数据，比如`#{1 3 4 45}`


In [2]:
(type '(1 2 3 4))
(type '[1 2 3 4])
(type {:1 2 :3 4})
(type #{1 2 3 4})

clojure.lang.PersistentList clojure.lang.PersistentVector clojure.lang.PersistentArrayMap clojure.lang.PersistentHashSet

对于可以操作数据的函数，可以从集合所实现的抽象接口来整理。clojure为collection定义了如下常用的抽象接口
- collection
- Sequence
- Associative
- Indexed
- Stack
- Set
- Sorted

### collection 集合抽象接口

所有的list, map, set, vector都实现了collection抽象接口， collection实现了如下函数.
- conj: conjoin，添加元素到集合
- seq: 获得顺序视图
- count：获得元素个数
- empty: 获得和集合一样的空集合
- =：判断是否相等


conj用来将元素添加到一个集合之中，对于不同的集合类型，其操作效果并不一样。

In [2]:
(conj '(1 2 3 4) 6 7)     ; insert in front of a list
(conj [1 2 3 4] 6)        ; append to vector
(conj #{1 2 3 4} 6)       ; set has no order
(conj {:a 1 :b 2} {:c 2}) ; insert a key value pair

(7 6 1 2 3 4) [1 2 3 4 6] #{1 4 6 3 2} {:a 1, :b 2, :c 2}

conj之所以将元素添加到list的头部，是因为如果添加到尾部，需要遍历整个列表，效率非常低下。
由conj衍生出来一个函数叫做`into`，它能将不同类型集合合并到第一个参数的集合中去(返回类型为第一个参数类型)

In [3]:
(into [1 2 3] '(4 5 6))
(into '(1 2 3 4) [5 6 7 8])

[1 2 3 4 5 6] (8 7 6 5 1 2 3 4)

empty函数可以保证创建一个相同类型的数据。有时候我们写一个函数的时候，可能不知道传入参数的类型，此时就可以使用empty来保证输出和输入相同。比如如下 `map-map`函数对map的值进行操作。

In [4]:
(defn map-map
  [f m]
  (into (empty m)
        (for [[k v] m]
          [k (f v)])))
(map-map inc (hash-map :z 5 :c 6 :a 0))
(map-map inc (sorted-map :z 5 :c 6 :a 0))

#'user/map-map {:z 6, :c 7, :a 1} {:a 1, :c 7, :z 6}

### Sequence 序列 
序列抽象接口定义了遍历集合的元素的方法

- seq 函数返回传入参数的一个序列
- first, rest, next 提供了遍历序列的方法
- lazy-seq

first, reset的作用很明显。next和rest区别在于 next对于空序列返回nil，而rest返回空序列

In [7]:
(next ())
(rest ())
(next nil)
(rest nil)

nil () nil ()

需要注意的一点是**序列不是列表**(但列表是序列)。它们的主要区别在于
- 计算序列的长度比较耗时
- 序列可能是惰性的，只有真正用到它的时候才会计算值。
- 序列可能是无限长的，也就是不可数的
- 列表保存了长度信息，可以高效的获得

一般我们不需要主动创建序列，而是通过其他函数间接来调用。如果需要，有两个方法：`cons`和`list*`。cons将一个参数加入到一个集合的头部并返回一个序列。


In [15]:
(type (range 1 5))
(type (cons 0 (range 1 5)))
(type (list* 1 2 3 4 (range 1)))

clojure.lang.LongRange clojure.lang.Cons clojure.lang.Cons

`lazy-seq`用来创建一个惰性序列。比如如下函数

In [3]:
(defn random-limit 
  [limit]
  (lazy-seq (println "init number")(cons (rand-int limit) (random-limit limit))))
(take 10 (random-limit 50))

#'user/random-limit (44 5 17 23 49 8 35 29 16 34)

显然randomlimit是一个无限递归的函数。我们使用lazy-seq可以将其真正求值推迟到使用的时候。

`next`和`rest`在处理惰性列表的时候行为不相同。当使用`next`的时候，它会对列表尾巴的第一个元素进行求值，而`rest`只会简单地返回惰性列表。而且使用let解构的时候，始终使用`next`。

In [5]:
(def x (next (random-limit 10)))
(def y (rest (random-limit 10)))
(let [[x & others] (random-limit 10)])

#'user/x #'user/y nil

### Associate接口

Associate接口抽象的是把key value关联起来的数据方法。map/vector支持Associate接口，包括
- assoc  ：添加新的key value
- dissoc ：删除key对应的键值对
- get    ：获得key对应的值
- contains? ：是否包含对应的key？

在数据类型为vector的情况下，key对应的就是vector的下标。

In [3]:
(assoc {} 2 3 4 5)
(assoc [1 3 6] 2 :3)   ; have to pass index
(dissoc {1 4 5 7} 5)
(get [:4 3 2] 0)
(get {1 3 4 :5} 4)
(get {} :a "not found")
(get {} "No such key")
(get {"No such key" nil} "No such key")
(contains? {:k :v} :k)

{2 3, 4 5} [1 3 :3] {1 4} :4 :5 "not found" nil nil true

对于`get`，没有key的时候会返回`nil`，但是如果对应值也是`nil`，也返回同样的`nil`。这是一个需要注意的问题。为了避免这种情况的出现，我们可以使用`find`，其返回的是键值对。

In [1]:
(find {:k nil :b 2} :k)

[:k nil]

### Indexed接口
index接口下只有一个函数`nth`，用来获得一个下标对应的元素，当出现越界的时候抛出异常。

In [5]:
(nth [1 4] 1)

4

### stack 接口
stack就是栈，支持后进先出。它以下通过三个操作支持栈接口
- conj 添加
- pop 移除头部
- peek 查看头部

In [8]:
(pop [0 1 2 3 4])
(pop '(0 1 2 3 4))
(peek [1 2 4 4])

[0 1 2 3] (1 2 3 4) 4

### Sorted接口

sorted接口保证集合内的元素被以特定的顺序保存
- rseq返回一个逆序集合
- subseq返回一个区间内的集合
- rsubseq返回一个区间内的逆序集合

只有`map`和`set`实现了sorted接口，可以使用`sorted-map`和`sorted-set`来创建对应的数据

In [7]:
(def m (sorted-map :b 1 :a 2 :c 4))
m
(rseq m)
(subseq m <= :b)
(rsubseq m >= :b)

#'user/m {:a 2, :b 1, :c 4} ([:c 4] [:b 1] [:a 2]) ([:a 2] [:b 1]) ([:c 4] [:b 1])

## 访问元素的一般方法
一些clojure的数据结构都可以当成函数来使用，包括map，vector和set。我们将key传给这个数据，从而获得对应的值。对于map，我们还可以设定没有找到key时候的默认值，但是对于vector不能这么做。

In [10]:
({:a 1 :b 2} :a)
([:a :b :c] 1)
(#{:a :b :c} :c)
({:a 1 :b 2} :c 3)

1 :b :c 3

有时候key也可以当成函数使用,比如

In [13]:
(:a {:a 2 :b 3})

2